<a href="https://colab.research.google.com/github/Mayaa722/Ai-safety-system/blob/main/Fights_dataset_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium undetected-chromedriver
!pip install selenium

In [ ]:
!apt-get update
!apt-get install -y wget unzip
!apt-get remove -y chromium-chromedriver

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 384 kB in 2s (164 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https

In [ ]:
!pip install webdriver-manager selenium


In [ ]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y

--2025-11-28 00:48:35--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 74.125.202.93, 74.125.202.136, 74.125.202.91, ...
Connecting to dl.google.com (dl.google.com)|74.125.202.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117885136 (112M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb.2’

google-chrome-stabl 100%[===================>] 112.42M   245MB/s    in 0.5s    

2025-11-28 00:48:36 (245 MB/s) - ‘google-chrome-stable_current_amd64.deb.2’ saved [117885136/117885136]

(Reading database ... 122141 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (142.0.7444.175-1) over (142.0.7444.175-1) ...
Setting up google-chrome-stable (142.0.7444.175-1) ...
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package l

In [ ]:
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

cp: cannot stat '/usr/lib/chromium-browser/chromedriver': No such file or directory


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.binary_location = "/usr/bin/google-chrome-stable"


In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)


In [ ]:
driver.get("https://www.google.com")
print(driver.title)
driver.quit()

Google


In [ ]:
import os
import time
import requests
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException

In [ ]:
def scrape_fight_images(queries, target_count=150, pages=2, delay=2, download_folder="fight_images"):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    downloaded_urls = set()
    count = 0

    for query in queries:
        query_url = query.replace(" ", "+")
        for page in range(1, pages + 1):
            url = f"https://www.bing.com/images/search?q={query_url}&first={(page-1)*35+1}"
            try:
                driver.get(url)
            except Exception as e:
                print(f"Page load failed, skipping page: {e}")
                continue

            time.sleep(delay)
            images = driver.find_elements(By.TAG_NAME, "img")

            for img in images:
                # Get both src and data-src (lazy-loaded)
                src = img.get_attribute("src") or img.get_attribute("data-src")
                if src and src.startswith("http") and src not in downloaded_urls:
                    downloaded_urls.add(src)
                    try:
                        response = requests.get(src)
                        if response.status_code == 200:
                            count += 1
                            file_path = os.path.join(download_folder, f"{query.replace(' ', '_')}_{count}.jpg")
                            with open(file_path, "wb") as f:
                                f.write(response.content)
                            print(f"Downloaded {file_path}")

                        if count >= target_count:
                            print(f"Reached target of {target_count} images")
                            return list(downloaded_urls)

                    except Exception as e:
                        print(f"Failed to download {src}: {e}")

    print(f"Total images downloaded: {count}")
    return list(downloaded_urls)

In [ ]:
fight_queries = [
    "street fight",
    "people fighting",
    "physical fight",
    "fist fight",
    "crowd fight",
    "assault fight",
    "attack fight",
    "real fight",
    "violent fight"
]

results = scrape_fight_images(
    queries=fight_queries,
    target_count=300,
    pages=2,
    delay=3,
    download_folder="fight_images"
)

print(f"Total images downloaded: {len(results)}")

Page load failed, skipping page: HTTPConnectionPool(host='localhost', port=55355): Max retries exceeded with url: /session/3618dfa2e327b9ef206ba5e7bce37e2f/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f58f9a8d220>: Failed to establish a new connection: [Errno 111] Connection refused'))
Page load failed, skipping page: HTTPConnectionPool(host='localhost', port=55355): Max retries exceeded with url: /session/3618dfa2e327b9ef206ba5e7bce37e2f/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f58f9a8dca0>: Failed to establish a new connection: [Errno 111] Connection refused'))
Page load failed, skipping page: HTTPConnectionPool(host='localhost', port=55355): Max retries exceeded with url: /session/3618dfa2e327b9ef206ba5e7bce37e2f/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f58f9c43dd0>: Failed to establish a new connection: [Errno 111] Connection refused'))
Page load failed, skippin

In [ ]:
!ls fight_images


fight_1.jpg


In [ ]:
from google.colab import files
import shutil

In [ ]:
folder_path = "fight_images"
zip_path = "/content/fight_images.zip"

In [ ]:
shutil.make_archive("/content/fight_images", 'zip', folder_path)
zip_path

In [ ]:
files.download("/content/fight_images.zip")
